In [1]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
loadvar()

/home/jovyan/agave
MACHINE_IP=149.165.157.140
MACHINE_NAME=jetstream
DOMAIN=jetstream-cloud.org
EMAIL=sbrandt@cct.lsu.edu
AGAVE_USERNAME=stevenrbrandt
MACHINE_USERNAME=jovyan
BASE_APP_NAME=funwave-tvd
PORT=10022
DOCKERHUB_NAME=stevenrbrandt
WORK_DIR=/home/jovyan
HOME_DIR=/home/jovyan
SCRATCH_DIR=/home/jovyan
DEPLOYMENT_PATH=agave-deployment
PATH=/home/jovyan/agave/cli/bin:/opt/conda/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/jq/bin:/home/jovyan/src/cli/bin
EXEC_MACHINE=jetstream-exec-stevenrbrandt
AGAVE_JSON_PARSER=jq
APP_NAME=funwave-tvd-jetstream-stevenrbrandt
STORAGE_MACHINE=jetstream-storage-stevenrbrandt
REMOTE_COMMAND=cd funwave-tvd-docker && docker build --rm -t funwave-tvd .
IN_FILE=agave://jetstream-storage-stevenrbrandt/input.txt
PBTOK=.....
AGAVE_PASSWD=.....
JOB_ID=8722401213345501671-242ac11b-0001-007
STAT=PENDING


In [2]:
writefile("entrypoint.sh","""
#!/bin/bash
useradd --shell /bin/bash -u \$USER_ID -o -p foobar -c "" -m user
export HOME=/home/user
echo ARGS: \$@
id

exec su - user -c "\$@"
""")

Writing file `entrypoint.sh'


In [19]:
writefile("funwave-tvd-docker/Dockerfile","""
FROM stevenrbrandt/science-base

MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>
RUN cd && \
    git clone https://github.com/fengyanshi/FUNWAVE-TVD && \
    cd FUNWAVE-TVD/src && \
    perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile && \
    make

WORKDIR /home/jovyan/FUNWAVE-TVD/src
RUN mkdir -p /home/jovyan/FUNWAVE-TVD/rundir
WORKDIR /home/jovyan/FUNWAVE-TVD/rundir
USER root
COPY entrypoint.sh /usr/local/bin
RUN chmod 755 /usr/local/bin/entrypoint.sh
ENTRYPOINT ["/usr/local/bin/entrypoint.sh"]
""")

Writing file `funwave-tvd-docker/Dockerfile'


In [20]:
!tar czf dockerjob.tgz Dockerfile entrypoint.sh
!files-mkdir -S ${STORAGE_MACHINE} -N funwave-tvd-docker
!files-upload -F dockerjob.tgz -S ${STORAGE_MACHINE} funwave-tvd-docker/

Successfully created folder funwave-tvd-docker
Uploading dockerjob.tgz...
######################################################################## 100.0%


In [5]:
from runagavecmd import *

In [21]:
runagavecmd(
    "tar xzf dockerjob.tgz && sudo docker build --rm -t funwave-tvd .",
    "agave://${STORAGE_MACHINE}/funwave-tvd-docker/dockerjob.tgz"
)

REMOTE_COMMAND=tar xzf dockerjob.tgz && sudo docker build --rm -t funwave-tvd .
IN_FILE=agave://jetstream-storage-stevenrbrandt/funwave-tvd-docker/dockerjob.tgz
Writing file `job-remote.txt'
OUTPUT=Successfully submitted job 3636206933178323431-242ac11b-0001-007
JOB_ID=3636206933178323431-242ac11b-0001-007
STAT=PENDING
STAT=PENDING
STAT=PENDING
STAT=STAGED
STAT=STAGED
STAT=SUBMITTING
STAT=RUNNING
STAT=FINISHED
CMD=jobs-output-get 3636206933178323431-242ac11b-0001-007 fork-command-1.out
All done! Output follows.
Reading file `fork-command-1.out'
Sending build context to Docker daemon  13.82kB

Step 1/10 : FROM stevenrbrandt/science-base
 ---> e93db47f971c
Step 2/10 : MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>
 ---> Using cache
 ---> 9bcc1f7703a9
Step 3/10 : RUN cd &&     git clone https://github.com/fengyanshi/FUNWAVE-TVD &&     cd FUNWAVE-TVD/src &&     perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile &&     make
 ---> Using cache
 ---> 4b17dc8d3dbc
Step 4/10 : WORKDIR /home/

In [22]:
!jobs-output-get ${JOB_ID} fork-command-1.err
!cat fork-command-1.err

In [23]:
!jobs-output-list ${JOB_ID}

.agave.archive
.agave.log
Dockerfile
dockerjob.tgz
entrypoint.sh
fork-command-1.err
fork-command-1.ipcexe
fork-command-1.out
fork-command-1.pid
fork-test.txt
fork-wrapper.txt
